In [1]:
%%capture
from tqdm import tqdm
from convokit import Corpus, Speaker, Utterance
from collections import defaultdict
import pandas as pd
import pprint as pp
from IPython.display import display
from modules.DataPreprocesser import DataPreprocesser
from modules import CorpusUtils as corp
import torch

# Add the src directory to the path
import sys
import os
# Add the src

sys.path.append(os.path.abspath("."))
import import_ipynb
import data_preprocessing
#from modules.CorpusUtils import CorpusUtils

# Corpus Converter

This notebook takled the pre-processed dataframes and converts it to a Corpus Object

# Verifying the Corpus Dataframes

### Final Filtered Conversations and Utterances

In [2]:
filepath = "/Users/mishkin/Desktop/Research/Convo_Kit/ConvoKit_Disputes/data/preprocessed_dyads.csv"

final_data = DataPreprocesser(filepath)
final_data.getDataframe()

2200


,b_country,b_RankBuy_1,b_RankBuy_2,b_RankBuy_3,b_RankBuy_4,b_Tact_1,b_Tact_2,b_Tact_3,b_Tact_4,b_Tact_5,...,s_AI4u,s_PlayAI.1,s_AI-cues.1,is_AI,formattedChat,b_insight,s_insight,flag_speaker,dispute_outcome,parsed_dialog
0,187.0,80.0,12.0,5.0,3.0,1.0,4.0,5.0,NaN,1.0,...,NaN,NaN,NaN,True,nan Seller: Your sudden demand for a refund is...,NaN,NaN,0,0,"[{'timestamp': 'nan', 'speaker': 'Seller', 'me..."
1,187.0,40.0,10.0,31.0,19.0,3.0,1.0,2.0,NaN,1.0,...,NaN,NaN,NaN,False,1699388451 Buyer: Hi there! I believe I receiv...,3.0,1.0,3,1,"[{'timestamp': 1699388451, 'speaker': 'Buyer',..."
2,187.0,50.0,8.0,30.0,12.0,2.0,5.0,4.0,NaN,4.0,...,NaN,NaN,NaN,False,1699389969 Buyer: Hello. I need you to make th...,2.0,6.0,3,1,"[{'timestamp': 1699389969, 'speaker': 'Buyer',..."
3,187.0,80.0,0.0,20.0,0.0,1.0,3.0,4.0,NaN,4.0,...,NaN,NaN,NaN,False,1699392080 Buyer: I would prefer a refund as t...,2.0,5.0,1,1,"[{'timestamp': 1699392080, 'speaker': 'Buyer',..."
4,187.0,40.0,30.0,20.0,10.0,2.0,3.0,3.0,NaN,4.0,...,NaN,NaN,NaN,False,1699392404 Buyer: I didn't receive the correct...,NaN,NaN,3,1,"[{'timestamp': 1699392404, 'speaker': 'Buyer',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,161.0,15.0,20.0,25.0,40.0,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,because he responded genuinely,False,1718875387 Buyer: Hello i requesting for a par...,2.0,6.0,3,1,"[{'timestamp': 1718875387, 'speaker': 'Buyer',..."
2196,137.0,90.0,0.0,10.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,how he responded,False,"1718875522 Buyer: Hello, I would like to recei...",0.0,1.0,2,0,"[{'timestamp': 1718875522, 'speaker': 'Buyer',..."
2197,185.0,75.0,0.0,25.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,"1718875699 Buyer: I would like a refund, the b...",NaN,NaN,1,1,"[{'timestamp': 1718875699, 'speaker': 'Buyer',..."
2198,161.0,35.0,20.0,30.0,15.0,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,sentences were short and minimal information g...,False,"1718876074 Buyer: Hey, I received my order and...",4.0,3.0,3,1,"[{'timestamp': 1718876074, 'speaker': 'Buyer',..."


In [3]:
final_data.getUtterancesDF()

,timestamp,speaker,message,uttidx,speaker_id,is_AI,row_idx,match_idx,Case Match Type,convo_len
0,nan,Seller,Your sudden demand for a refund is unwarranted...,0,Seller_0,True,0,False,None,16
1,1699388150,Buyer,"This was not the item I ordered, please provid...",1,Buyer_0,False,0,False,None,16
2,1699388170,Seller,How about a compromise? I can offer you a part...,2,Seller_0,True,0,False,None,16
3,1699388217,Buyer,"No, this was not the item I ordered. I have a ...",3,Buyer_0,False,0,False,None,16
4,1699388235,Seller,"I can scrutinize your claim, but this requires...",4,Seller_0,True,0,False,None,16
...,...,...,...,...,...,...,...,...,...,...
29276,1718879513,Buyer,"Cool no problem, so just to clarify, you will ...",6,Buyer_2199,False,2199,False,None,11
29277,1718879641,Seller,I will refund you after you have sent the item...,7,Seller_2199,False,2199,False,None,11
29278,1718879680,Buyer,"Great thanks for this outcome, enjoy the rest ...",8,Buyer_2199,False,2199,False,None,11
29279,1718879753,Seller,"Submitted agreement: Buyer gets full refund, s...",9,Seller_2199,False,2199,False,None,11


### Corpus-converted Dataframes

In [4]:
test_utterances = corp.buildUtteranceDF(final_data.getUtterancesDF())
display(test_utterances)
#corp.setUtteranceMetadata(["dispute_outcome"])
test_spk = corp.buildSpeakerDF(final_data.getUtterancesDF())
display(test_spk)
test_conv = corp.buildConvoDF(final_data.getDataframe())
display(test_conv)

,id,speaker,conversation_id,reply_to,timestamp,text
0,utt0_con0,Seller_0,utt0_con0,None,nan,Your sudden demand for a refund is unwarranted...
1,utt1_con0,Buyer_0,utt0_con0,utt0_con0,1699388150,"This was not the item I ordered, please provid..."
2,utt2_con0,Seller_0,utt0_con0,utt1_con0,1699388170,How about a compromise? I can offer you a part...
3,utt3_con0,Buyer_0,utt0_con0,utt2_con0,1699388217,"No, this was not the item I ordered. I have a ..."
4,utt4_con0,Seller_0,utt0_con0,utt3_con0,1699388235,"I can scrutinize your claim, but this requires..."
...,...,...,...,...,...,...
29276,utt6_con2199,Buyer_2199,utt0_con2199,utt5_con2199,1718879513,"Cool no problem, so just to clarify, you will ..."
29277,utt7_con2199,Seller_2199,utt0_con2199,utt6_con2199,1718879641,I will refund you after you have sent the item...
29278,utt8_con2199,Buyer_2199,utt0_con2199,utt7_con2199,1718879680,"Great thanks for this outcome, enjoy the rest ..."
29279,utt9_con2199,Seller_2199,utt0_con2199,utt8_con2199,1718879753,"Submitted agreement: Buyer gets full refund, s..."


,id
0,Seller_0
1,Buyer_0
2,Buyer_1
3,Seller_1
4,Buyer_2
...,...
4395,Seller_2197
4396,Buyer_2198
4397,Seller_2198
4398,Buyer_2199


,id
0,0
1,1
2,2
3,3
4,4
...,...
2195,2195
2196,2196
2197,2197
2198,2198


# Creating Corpus Object

In [5]:
test_corp= corp.corpusBuilder(final_data)

29281it [00:00, 58137.24it/s]


In [6]:
test_corp.print_summary_stats()

Number of Speakers: 4400
Number of Utterances: 29281
Number of Conversations: 2200


In [7]:
display(test_corp.get_conversations_dataframe())
display(test_corp.get_utterances_dataframe())
display(test_corp.get_speakers_dataframe())

,vectors
id,
utt0_con0,[]
utt0_con1,[]
utt0_con2,[]
utt0_con3,[]
utt0_con4,[]
...,...
utt0_con2195,[]
utt0_con2196,[]
utt0_con2197,[]


,timestamp,text,speaker,reply_to,conversation_id,vectors
id,,,,,,
utt0_con0,nan,Your sudden demand for a refund is unwarranted...,Seller_0,None,utt0_con0,[]
utt1_con0,1699388150,"This was not the item I ordered, please provid...",Buyer_0,utt0_con0,utt0_con0,[]
utt2_con0,1699388170,How about a compromise? I can offer you a part...,Seller_0,utt1_con0,utt0_con0,[]
utt3_con0,1699388217,"No, this was not the item I ordered. I have a ...",Buyer_0,utt2_con0,utt0_con0,[]
utt4_con0,1699388235,"I can scrutinize your claim, but this requires...",Seller_0,utt3_con0,utt0_con0,[]
...,...,...,...,...,...,...
utt6_con2199,1718879513,"Cool no problem, so just to clarify, you will ...",Buyer_2199,utt5_con2199,utt0_con2199,[]
utt7_con2199,1718879641,I will refund you after you have sent the item...,Seller_2199,utt6_con2199,utt0_con2199,[]
utt8_con2199,1718879680,"Great thanks for this outcome, enjoy the rest ...",Buyer_2199,utt7_con2199,utt0_con2199,[]


,vectors
id,
Seller_0,[]
Buyer_0,[]
Buyer_1,[]
Seller_1,[]
Buyer_2,[]
...,...
Seller_2197,[]
Buyer_2198,[]
Seller_2198,[]


# Adding MetaData with ConvoKit 
- Need to add dispuet outcome labels

# Testing
## Notes
- ConvoKit's Forecaster is a modular implementation of general forecasting behavior, meaning that it does not implement any specific forecasting algorithm and instead encapsulates the general behavior of iterating over a conversation and producing a forecast score upon each new comment (using some specified algorithm).
- seem to be able to use .transform() to make predicitons on corpus object if we import pre-trained CRAFT model.
- The CRAFT model needs tokenized and processed conversation data. Funciton exists in Convokit
- wiki_awry_demo: Transformers: ExpectedContextModel, Prompt Types, politeness strategies
- craft-demo-new: best model to learn since uses pre-trained Wikiconv-based model as reported in the "Trouble on the Horizon" paper

# First Step Todo

1. craft_demo_new to make predictions with forecaster using corpus object
2. How to tokenize our data to give as input to model and make predictions (no fine-tuning, just inference)
3. Where do I add conversation outcome labels?

# Testing Predictions

### Add Conversation Labels

In [8]:
for idx, row in final_data.getDataframe().iterrows():
    convo_id = f"utt0_con{idx}"  # generate conversation_id format from index
    label = row["dispute_outcome"]  # update if your label column is named differently
    if convo_id in test_corp.conversations:
        test_corp.get_conversation(convo_id).meta["label"] = label

### Testing Label Output

In [9]:
for convo in test_corp.iter_conversations():
    print(f"Conversation ID: {convo.id}")
    print("Metadata:", convo.meta)
    print("---")

Conversation ID: utt0_con0
Metadata: ConvoKitMeta({'label': 0})
---
Conversation ID: utt0_con1
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con2
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con3
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con4
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con5
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con6
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con7
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con8
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con9
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con10
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con11
Metadata: ConvoKitMeta({'label': 0})
---
Conversation ID: utt0_con12
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con13
Metadata: ConvoKitMeta({'label': 1})
---
Conversation ID: utt0_con14
Metadata: ConvoK

### Importing CRAFT Model

In [10]:
from convokit.forecaster.CRAFTModel import CRAFTModel

model = CRAFTModel(
    initial_weights="craft-wiki-pretrained",  # or "craft-wiki-finetuned"
    torch_device="cuda" if torch.cuda.is_available() else "cpu"
)

### Importing Forcaster Object

In [11]:
from convokit.forecaster.forecaster import Forecaster

# Use 'label' because we added it to conversation.meta["label"]
forecaster = Forecaster(
    forecaster_model=model,
    labeler="label",  # string, not function
    forecast_attribute_name="my_forecast",
    forecast_prob_attribute_name="my_forecast_prob"
)

# This will train and predict (if pretrained, "fit" is actually just transformation)
test_corp = forecaster.fit_predict(test_corp)


AttributeError: 'Forecaster' object has no attribute 'fit_predict'

### Look at the predictions

In [ ]:
for convo in corpus.iter_conversations():
    print("ID:", convo.id)
    print("Predicted:", convo.meta.get("my_forecast"))
    print("Probability:", convo.meta.get("my_forecast_prob"))
    print("Actual Label:", convo.meta.get("label"))
    print("----")

### Fixing Timestamps


In [22]:
for utt in test_corp.iter_utterances():
    ts = utt.timestamp
    try:
        utt.timestamp = int(ts) if ts is not None and str(ts).lower() != 'nan' else None
    except (ValueError, TypeError):
        utt.timestamp = None
        
        
for utt in test_corp.iter_utterances():
    assert isinstance(utt.timestamp, (int, type(None)))

# Example: Get all utterance IDs
utterance_ids = list(test_corp.utterances.keys())
# Access one utterance
utt = test_corp.utterances[utterance_ids[0]]

# Print the timestamp
print("Timestamp:", utt.timestamp)


# Create a list of utterance metadata
data = [
    {
        "utterance_id": utt.id,
        "speaker": utt.speaker.id if utt.speaker else None,
        "conversation_id": utt.conversation_id,
        "timestamp": utt.timestamp,
        "text": utt.text
    }
    for utt in test_corp.iter_utterances()
]

# Convert to DataFrame
df = pd.DataFrame(data)
# Convert timestamps to float (optional if inconsistent)
df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce')

# Sort by timestamp
df_sorted = df.sort_values('timestamp')

# Preview
display(df)


Timestamp: None


,utterance_id,speaker,conversation_id,timestamp,text
0,utt0_con0,Seller_0,utt0_con0,NaN,Your sudden demand for a refund is unwarranted...
1,utt1_con0,Buyer_0,utt0_con0,1.699388e+09,"This was not the item I ordered, please provid..."
2,utt2_con0,Seller_0,utt0_con0,1.699388e+09,How about a compromise? I can offer you a part...
3,utt3_con0,Buyer_0,utt0_con0,1.699388e+09,"No, this was not the item I ordered. I have a ..."
4,utt4_con0,Seller_0,utt0_con0,1.699388e+09,"I can scrutinize your claim, but this requires..."
...,...,...,...,...,...
29276,utt6_con2199,Buyer_2199,utt0_con2199,1.718880e+09,"Cool no problem, so just to clarify, you will ..."
29277,utt7_con2199,Seller_2199,utt0_con2199,1.718880e+09,I will refund you after you have sent the item...
29278,utt8_con2199,Buyer_2199,utt0_con2199,1.718880e+09,"Great thanks for this outcome, enjoy the rest ..."
29279,utt9_con2199,Seller_2199,utt0_con2199,1.718880e+09,"Submitted agreement: Buyer gets full refund, s..."


In [ ]:
import torch
from convokit.forecaster.CRAFTModel import CRAFTModel
from convokit.forecaster.forecaster import Forecaster
from convokit.forecaster.CRAFT.data import tokenize

# -----------------------------
# Step 2: Tokenize utterances (optional but efficient)
# -----------------------------
voc = model._voc  # CRAFT vocab for tokenizing

for utt in test_corp.iter_utterances():
    if "craft_tokens" not in utt.meta:
        utt.meta["craft_tokens"] = tokenize(voc, utt.text)

# -----------------------------
# Step 3: Define selector for prediction context (last utterance in each conversation)
# -----------------------------
def is_last_context(context_tuple):
    return len(context_tuple.future_context) == 0

# -----------------------------
# Step 4: Initialize Forecaster
# -----------------------------
forecaster = Forecaster(
    forecaster_model=model,
    labeler="label",  # assumes your conversations already have meta["label"]
    forecast_attribute_name="prediction",
    forecast_prob_attribute_name="pred_score"
)

# -----------------------------
# Step 5: Run predictions
# -----------------------------
# If model is pretrained only: just transform
test_corp = forecaster.transform(test_corp, context_selector=is_last_context)

# -----------------------------
# Step 6: Display results
# -----------------------------
for convo in list(test_corp.iter_conversations())[:5]:
    print("Conversation ID:", convo.id)
    print("Actual Label:", convo.meta.get("label"))
    print("Predicted Label:", convo.meta.get("prediction"))
    print("Confidence Score:", convo.meta.get("pred_score"))
    print("---")

# -----------------------------
# (Optional) Step 7: Export forecasts to DataFrame
# -----------------------------
forecast_df = forecaster.summarize(test_corp)
print(forecast_df.head())


ValueError: '<' not supported between instances of 'int' and 'NoneType'
Utterance timestamps may not have been set correctly.